In [4]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns

from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier

from sklearn.metrics import accuracy_score, recall_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV


from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

## 1. Выводы и наблюдения по результатам проведенной работы

Традиционно для удобства проверяющих общие соображения и выводы будут размещены здесь, в самом начале ноутбука. Код же в полном объеме можно найти во втором разделе данной работы, он последовательно разделён на логические этапы, но содержит минимум комментариев.

### 1.1. Понимание задачи и определение подходов к её решению

1. На предыдущем этапе в ходе работы в ходе EDA мы поняли, что решаем задачу бинарной классификации: совершит ли посетитель сайта покупку или нет.
2. На основе анализа распределения целевой переменной мы понимаем, что данные несбалансированные: только 16% посетителей сайта совершают покупку (что естественно).
3. На основании этих вводных я считаю правильным при любом разбиении данных для построения моделей задавать параметр stratify, чтобы распределение категорий при сплите было пропорциональным распределению категорий в общем датасете.
4. В качестве основной метрики качества я буду использовать recall. Я считаю, что при таком типе задач (совершит ли посетитель покупку в интернете или нет), основной бизнес-целью является увеличение числа покупающих пользователей. Бизнес заинтересован выявлять всех потенциальных покупателей и стимулировать их к покупке рекламой или иными методами. Таким образом его в первую очередь будет интересовать в большей степени охват моделью потенциальных покупателей (recall), а не точность  детектирования (precision). Я не использовал f1-меру по этой же причине. И понятно почему при такой трактовке задачи и несбалансированности классов совершенно бессмысленно метрикой качества модели определять accuracy.

### 1.2. Построение моделей на числовых признаках

В данном разделе я ставил целью выбор лучших моделей для решения задачи. Мне не требовалось строить окончательную точную модель, поэтому для ускорения не использовалась валидация.
В качестве набора экспериментальных моделей помимо Наивного Байеса (NB) и KNN я использовал CatBoostClassifier (CB), DecisionTreeClassifier (DT), RandomForestClassifier (RF) и GradientBoostingClassifier (GB). Каждая из моделей была применена для решения задачи с параметрами по умолчанию (Default), к большишнству из моделей была предпринята попытка подбора параметров c помощью GreedSearch (Grid). Итоговые результаты метрики recall моих экспериментов приведены в таблице ниже: 

Mod Default Grid
KNN 0.287   0.353
CB  0.525   0.525
RF  0.509   0.518
GB  0.538   0.529
DT  0.486   0.559
NB  0.410   0.410

Лучше всего себя показали бустинги (CatBoost и GradientBoosting), а также RandomForest, было принято решение попробовать их и при полном обучении с добавлением категориальных признаков. Очень сильно удалось поднять подбором гиперпараметров качество решающего дерева, но она не догнала модели-лидеры. Стоит так же отметить, что модель Градиентного Бустинга по умолчанию дала лучший результат, чем при подборе параметров по сетке. CatBoosting я не настраивал по сетке, так как ещё до этого убедился, что параметры по умолчанию для данной модели работают очень хорошо. 

### 1.3. Модели-лидеры на полных данных (добавление категориальных признаков)

Итак, были проведены эксперименты с бустингами и случайным лесом. При обучении моделей Случайного Леса и Градиентного бустинга и подборе гиперпараметров для них, был использован pipeline с OneHotEncoding для кодирования категориальных признаков. При обучении Catboost только передавались номера категориальных столбцов, так как данная модель умеет довольно хорошо кодировать категории автоматически под капотом. Можно было подобрать гиперпараметры CatBoost с помощью Optuna, но мне показалось это избыточным в рамках задач данного буткемпа. Результаты работы моделей на валидации, тесте и тесте после подбора гиперпараметров представлены в таблице:  

Mod Valid    Test Grid
CB  0.616   0.595   -
DT  0.625   0.588   0.565
GB  0.628   0.588   -

По сути модели показывают похожее качество, немного лучше на тестовых данных работает CatBoost с параметрами по умолчанию. При деплое будем использовать данную модель. Я хотел понять, как будет показывать себя обученная на recall модель с точки зрения accuracy. Оказалось, что наш лидер CatBoost показывает значение этой метрики в 0.893, что весьма неплохо.  

### 1.4. Explainer Dashboard

* С учетом того, что в качестве главной модели был выбран CatBoost с гиперпараметрами по умолчанию, строил ExplainerDashboard с помощью ClassifierExplainer с параметром model_output='logodds', как того требует документация.
* Готовый Explainer Dashboard был сохранён в файл dashboard.html в корневой директории текущего репозитория
* Самыми важными признаками с точки зрения Permutation Importances с большим опережением являются присвоенная ценность страницы (PageValues) и месяц, в который пользователь посещал сайт. Между этими признаками и целевой переменной наблюдается прямая корреляции. Менее всего влияют на целевую переменную признаки WeekDay и SpecialDay, хотя мне казалось, что даннные признаки должны быть очень важны. Признаки с обратной корреляцией являются тип браузера и операционной системы пользователя, их влияние на целевую переменную крайне незначительно.
* Важность признаков с точки зрения SHAP values по сравнению с Permutation Importances примерно такая же, но разрыв между PageValues и Month и иными признаками не такая огромная.
* Результаты метрик качества выводятся следующие: 
    metric	    Score
    accuracy	0.893
    precision	0.703
    recall	    0.595
    f1	        0.645
Мне кажется я уже описал своё понимание полученных метрик выше, и здесь дублировать эти мысли не буду
* Попробуем проинтерпретировать результаты в отношении пары случайных наблюдений:
Наблюдение 11344:
* С вероятностью в 69% пользователь с такими характеристиками не сделает покупку на сайте. Основной вес в этом решении модели оказывает фактор ценности страницы = 26.41113767 и месяца совершения покупки (ноябрь). То, что этот пользователь является "возвратившимся" и продолжительность его пребывания на страницах сайта с категорией "административной" увеличивает шанс на покупку таким пользователем.
Наблюдение 1721:
* С вероятностью в 99,5% пользователь с такими характеристиками сделает покупку на сайте. Основной вес в этом решении модели оказывает фактор ценности страницы и месяца совершения покупки (март). Только значения категорий ExitRates, TrafficTypes, SpecialDay и BounceRates сыграло против сделанного моделью прогноза, все остальные параметры только довносят свой вклад в уверенность модели и итоговый прогноз.

P.S. Оценивая отдельные результаты мне стало казаться, что я неправильно передал в Explainer названия лейблов (нарушил последовательность), и прогноз покупать/не покупать стоит интерпретировать строго наборот. Но я это делал согласно порядку, указанному в документации, как я его понял, поэтому уверенности в правильной итоговой интерпретации у меня нет.    
    

# 2. Решение задаче в коде

## 2.1. Чтение полученного по результатам EDA датасета, вспоминаем характеристики 

In [5]:
df = pd.read_pickle("df_eda.pickle")

In [7]:
df.sample(5)

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,y
10124,0,0.000,0,0.0,2,87.000000,0.000000,0.033333,0.000,0.0,Nov,3,2,4,11,Returning_Visitor,0,0
4452,0,0.000,0,0.0,19,160.500000,0.000000,0.042105,0.000,0.8,May,2,2,4,6,Returning_Visitor,0,0
3778,0,0.000,0,0.0,8,123.000000,0.000000,0.014286,59.988,0.0,May,2,2,1,2,New_Visitor,0,1
11601,18,514.075,4,67.5,178,6630.479365,0.003646,0.010805,0.000,0.0,Nov,3,2,1,8,Returning_Visitor,0,1
5888,1,49.400,0,0.0,18,213.171429,0.000000,0.003922,0.000,0.0,Oct,2,2,1,2,New_Visitor,0,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10855 entries, 1 to 12329
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Administrative           10855 non-null  int64  
 1   Administrative_Duration  10855 non-null  float64
 2   Informational            10855 non-null  int64  
 3   Informational_Duration   10855 non-null  float64
 4   ProductRelated           10855 non-null  int64  
 5   ProductRelated_Duration  10855 non-null  float64
 6   BounceRates              10855 non-null  float64
 7   ExitRates                10855 non-null  float64
 8   PageValues               10855 non-null  float64
 9   SpecialDay               10855 non-null  float64
 10  Month                    10855 non-null  object 
 11  OperatingSystems         10855 non-null  int64  
 12  Browser                  10855 non-null  int64  
 13  Region                   10855 non-null  int64  
 14  TrafficType              10

In [3]:
X = df[['Administrative_Duration', 'Informational_Duration', 'ProductRelated_Duration', 'BounceRates', 'ExitRates', 'PageValues']]

In [4]:
y = df['y']

In [13]:
X.shape, y.shape

((10855, 6), (10855,))

In [20]:
y.value_counts()

y
0    9088
1    1767
Name: count, dtype: int64

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

In [24]:
X_train.shape, X_test.shape

((8141, 6), (2714, 6))

## 2.2. Первое приближение: строим модели на accuracy

### Наивный байесовский классификатор

In [25]:
gnb = GaussianNB()

gnb.fit(X_train, y_train)

y_pred = gnb.predict(X_test)

In [26]:
accuracy_score(y_test, y_pred)

0.8581429624170965

In [27]:
confusion_matrix(y_test, y_pred)

array([[2148,  124],
       [ 261,  181]], dtype=int64)

### KNN

In [28]:
knn_cl = KNeighborsClassifier()

knn_cl.fit(X_train, y_train)

pred_knn = knn_cl.predict(X_test)

In [29]:
accuracy_score(y_test, pred_knn)

0.8496683861459101

In [30]:
confusion_matrix(y_test, pred_knn)

array([[2179,   93],
       [ 315,  127]], dtype=int64)

## Кросс-валидация

In [33]:
cross_val_score(knn_cl, X, y, scoring='accuracy', cv=3, n_jobs=-1).mean()

0.8530614303346388

In [32]:
cross_validate(knn_cl, X, y, scoring=['accuracy', 'f1'], cv=3, n_jobs=-1)

{'fit_time': array([0.01209068, 0.01209068, 0.01209068]),
 'score_time': array([0.13879037, 0.13879037, 0.13879037]),
 'test_accuracy': array([0.87123515, 0.84715312, 0.84079602]),
 'test_f1': array([0.45433255, 0.38073908, 0.30935252])}

In [34]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42, stratify=y_train)

In [35]:
best_k = -1
best_acc = -1

for k in np.arange(2, 30, 1):
    knn_cl = KNeighborsClassifier(n_neighbors=k) # по умолчанию n_neighbors=5

    knn_cl.fit(X_train, y_train)
    pred_knn = knn_cl.predict(X_val)
    acc = accuracy_score(y_val, pred_knn)

    if acc > best_acc:
        best_k = k
        best_acc = acc

    print('n_neighbors:', k, 'accuracy:', acc)

print(best_k, best_acc)

n_neighbors: 2 accuracy: 0.8526522593320236
n_neighbors: 3 accuracy: 0.8521611001964636
n_neighbors: 4 accuracy: 0.8546168958742633
n_neighbors: 5 accuracy: 0.8570726915520629
n_neighbors: 6 accuracy: 0.8614931237721022
n_neighbors: 7 accuracy: 0.8575638506876228
n_neighbors: 8 accuracy: 0.8585461689587426
n_neighbors: 9 accuracy: 0.8605108055009824
n_neighbors: 10 accuracy: 0.8595284872298625
n_neighbors: 11 accuracy: 0.8600196463654224
n_neighbors: 12 accuracy: 0.8595284872298625
n_neighbors: 13 accuracy: 0.8605108055009824
n_neighbors: 14 accuracy: 0.8575638506876228
n_neighbors: 15 accuracy: 0.8575638506876228
n_neighbors: 16 accuracy: 0.8565815324165029
n_neighbors: 17 accuracy: 0.8560903732809431
n_neighbors: 18 accuracy: 0.8565815324165029
n_neighbors: 19 accuracy: 0.8555992141453831
n_neighbors: 20 accuracy: 0.8536345776031434
n_neighbors: 21 accuracy: 0.8536345776031434
n_neighbors: 22 accuracy: 0.8526522593320236
n_neighbors: 23 accuracy: 0.8526522593320236
n_neighbors: 24 ac

In [36]:
knn_cl = KNeighborsClassifier(n_neighbors=6) # по умолчанию n_neighbors=5

knn_cl.fit(X_train, y_train)
pred_knn = knn_cl.predict(X_test)

acc = accuracy_score(y_test, pred_knn)
print(acc)

0.8566691230655858


In [37]:
confusion_matrix(y_test, pred_knn)

array([[2232,   40],
       [ 349,   93]], dtype=int64)

## 2.3. Акцент на recall - строим модели по умолчанию и подбираем параметры на GridSearch

### KNN

In [11]:
knn_cl = KNeighborsClassifier()

knn_cl.fit(X_train, y_train)

pred_knn = knn_cl.predict(X_test)

recall_score(y_test, pred_knn)

0.2873303167420814

In [12]:
cross_val_score(knn_cl, X, y, scoring='recall', cv=3, n_jobs=-1).mean()

0.279003961516695

In [13]:
cross_validate(knn_cl, X, y, scoring=['accuracy', 'recall', 'f1'], cv=3, n_jobs=-1)

{'fit_time': array([0.0060885 , 0.00261712, 0.0060885 ]),
 'score_time': array([0.13486457, 0.1347394 , 0.12680864]),
 'test_accuracy': array([0.87123515, 0.84715312, 0.84079602]),
 'test_recall': array([0.32937182, 0.28862479, 0.21901528]),
 'test_f1': array([0.45433255, 0.38073908, 0.30935252])}

In [16]:
model = KNeighborsClassifier()

params = {'n_neighbors' : np.arange(2, 20, 2),
          'weights' : ['uniform', 'distance'],
          'p': [1,2]}

gs = GridSearchCV(model, params, scoring='recall', cv=3, verbose=2)
gs.fit(X_train, y_train)

Fitting 3 folds for each of 36 candidates, totalling 108 fits
[CV] END ................n_neighbors=2, p=1, weights=uniform; total time=   0.0s
[CV] END ................n_neighbors=2, p=1, weights=uniform; total time=   0.0s
[CV] END ................n_neighbors=2, p=1, weights=uniform; total time=   0.0s
[CV] END ...............n_neighbors=2, p=1, weights=distance; total time=   0.0s
[CV] END ...............n_neighbors=2, p=1, weights=distance; total time=   0.0s
[CV] END ...............n_neighbors=2, p=1, weights=distance; total time=   0.0s
[CV] END ................n_neighbors=2, p=2, weights=uniform; total time=   0.0s
[CV] END ................n_neighbors=2, p=2, weights=uniform; total time=   0.0s
[CV] END ................n_neighbors=2, p=2, weights=uniform; total time=   0.0s
[CV] END ...............n_neighbors=2, p=2, weights=distance; total time=   0.0s
[CV] END ...............n_neighbors=2, p=2, weights=distance; total time=   0.0s
[CV] END ...............n_neighbors=2, p=2, wei

GridSearchCV(cv=3, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18]),
                         'p': [1, 2], 'weights': ['uniform', 'distance']},
             scoring='recall', verbose=2)

In [17]:
gs.best_score_, gs.best_params_

(0.39471344093192834, {'n_neighbors': 2, 'p': 2, 'weights': 'distance'})

In [19]:
pred = gs.best_estimator_.predict(X_test)

recall_score(y_test, pred)

0.35294117647058826

In [20]:
confusion_matrix(y_test, pred)

array([[2047,  225],
       [ 286,  156]], dtype=int64)

### CatBoost

In [24]:
!pip install catboost

  Using cached catboost-1.2.2-cp311-cp311-win_amd64.whl.metadata (1.2 kB)
  Using cached graphviz-0.20.1-py3-none-any.whl (47 kB)
  Using cached plotly-5.18.0-py3-none-any.whl.metadata (7.0 kB)
  Using cached tenacity-8.2.3-py3-none-any.whl.metadata (1.0 kB)
   ---------------------------------------- 0.0/101.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.0 MB 1.4 MB/s eta 0:01:14
   ---------------------------------------- 0.1/101.0 MB 1.7 MB/s eta 0:01:00
   ---------------------------------------- 0.4/101.0 MB 2.7 MB/s eta 0:00:38
   ---------------------------------------- 0.7/101.0 MB 3.4 MB/s eta 0:00:30
   ---------------------------------------- 0.8/101.0 MB 3.3 MB/s eta 0:00:31
   ---------------------------------------- 1.0/101.0 MB 3.9 MB/s eta 0:00:26
   ---------------------------------------- 1.0/101.0 MB 3.9 MB/s eta 0:00:26
   ---------------------------------------- 1.0/101.0 MB 3.9 MB/s eta 0:00:26
   ---------------------------------------- 1.

In [29]:
model_CB = CatBoostClassifier(random_state=42, verbose=0).fit(X_train, y_train)

y_pred = model_CB.predict(X_test)
recall_score(y_test, y_pred)

0.5248868778280543

### RandomForest

In [35]:
modelRF = RandomForestClassifier().fit(X_train, y_train)
y_pred = modelRF.predict(X_test)
recall_score(y_test, y_pred)

0.5090497737556561

In [36]:
params = {
          'max_depth' : np.arange(2,10),
          'min_samples_split' : np.arange(2,10,2),
          'min_samples_leaf': np.arange(1,10,2)}

gs = GridSearchCV(RandomForestClassifier(), params, cv=3, scoring='recall', verbose=0)

gs.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=RandomForestClassifier(),
             param_grid={'max_depth': array([2, 3, 4, 5, 6, 7, 8, 9]),
                         'min_samples_leaf': array([1, 3, 5, 7, 9]),
                         'min_samples_split': array([2, 4, 6, 8])},
             scoring='recall')

In [37]:
gs.best_score_, gs.best_params_

(0.5645403460529511,
 {'max_depth': 8, 'min_samples_leaf': 7, 'min_samples_split': 6})

In [38]:
modelRF_tuned = gs.best_estimator_.fit(X_train, y_train)
y_pred = modelRF_tuned.predict(X_test)
recall_score(y_test, y_pred)

0.5180995475113123

### GradientBoosting

In [36]:
modelGB = GradientBoostingClassifier().fit(X_train, y_train)
y_pred = modelGB.predict(X_test)
recall_score(y_test, y_pred)

0.5384615384615384

In [28]:
params = {'min_samples_leaf': np.arange(1,10,2)}

# 'max_depth' : np.arange(2,10)
# 'min_samples_split' : np.arange(2,10,2),
# 'min_samples_leaf': np.arange(1,10,2)

gs = GridSearchCV(GradientBoostingClassifier(max_depth=2, min_samples_split=2), params, cv=3, scoring='recall', verbose=0)

gs.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=GradientBoostingClassifier(max_depth=2),
             param_grid={'min_samples_leaf': array([1, 3, 5, 7, 9])},
             scoring='recall')

In [29]:
gs.best_score_, gs.best_params_

(0.5720955048686142, {'min_samples_leaf': 3})

In [33]:
modelGB_tuned = GradientBoostingClassifier(max_depth=2, min_samples_split=2, min_samples_leaf=3).fit(X_train, y_train)
y_pred = modelGB_tuned.predict(X_test)
recall_score(y_test, y_pred)

0.5294117647058824

### DecisionTree

In [38]:
modelDT = DecisionTreeClassifier().fit(X_train, y_train)
y_pred = modelDT.predict(X_test)
recall_score(y_test, y_pred)

0.48642533936651583

In [17]:
params = {'max_depth' : np.arange(2,20),
          'min_samples_split' : np.arange(2,20,2),
          'min_samples_leaf': np.arange(1,20,2)}

gs = GridSearchCV(DecisionTreeClassifier(), params, cv=3, scoring='recall', verbose=0)

gs.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19]),
                         'min_samples_leaf': array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19]),
                         'min_samples_split': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18])},
             scoring='recall')

In [18]:
gs.best_score_, gs.best_params_

(0.5592390802474836,
 {'max_depth': 13, 'min_samples_leaf': 17, 'min_samples_split': 16})

In [19]:
y_pred = gs.best_estimator_.predict(X_test)
recall_score(y_test, y_pred)

0.5407239819004525

### Наивный Баес

In [40]:
gnb = GaussianNB()

gnb.fit(X_train, y_train)

y_pred = gnb.predict(X_test)

recall_score(y_test, y_pred)

0.4095022624434389

Mod Default Grid
KNN 0.287   0.353
CB  0.525   0.525
RF  0.509   0.518
GB  0.538   0.529
DT  0.486   0.559
NB  0.410   0.410

=> CB, GB, DT

# 2.4. Добавляем категориальные признаки

In [6]:
X = df.drop(columns=['y', 'Administrative', 'Informational', 'ProductRelated'])

In [7]:
y = df['y']

In [44]:
X

,Administrative_Duration,Informational_Duration,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend
1,0.0,0.0,64.000000,0.000000,0.100000,0.000000,0.0,Feb,2,2,1,2,Returning_Visitor,0
3,0.0,0.0,2.666667,0.050000,0.140000,0.000000,0.0,Feb,3,2,2,4,Returning_Visitor,0
4,0.0,0.0,627.500000,0.020000,0.050000,0.000000,0.0,Feb,3,3,1,4,Returning_Visitor,1
5,0.0,0.0,154.216667,0.015789,0.024561,0.000000,0.0,Feb,2,2,1,3,Returning_Visitor,0
9,0.0,0.0,738.000000,0.000000,0.022222,0.000000,0.4,Feb,2,4,1,2,Returning_Visitor,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12325,145.0,0.0,1783.791667,0.007143,0.029031,12.241717,0.0,Dec,4,6,1,1,Returning_Visitor,1
12326,0.0,0.0,465.750000,0.000000,0.021333,0.000000,0.0,Nov,3,2,1,8,Returning_Visitor,1
12327,0.0,0.0,184.250000,0.083333,0.086667,0.000000,0.0,Nov,3,2,1,13,Returning_Visitor,1
12328,75.0,0.0,346.000000,0.000000,0.021053,0.000000,0.0,Nov,2,2,3,11,Returning_Visitor,0


In [45]:
X.columns

Index(['Administrative_Duration', 'Informational_Duration',
       'ProductRelated_Duration', 'BounceRates', 'ExitRates', 'PageValues',
       'SpecialDay', 'Month', 'OperatingSystems', 'Browser', 'Region',
       'TrafficType', 'VisitorType', 'Weekend'],
      dtype='object')

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42, stratify=y_train)

### 2.4.1 CatBoost

In [9]:
modelCB = CatBoostClassifier(verbose=0, cat_features=[7,8,9,10,11,12,13])

In [10]:
modelCB.fit(X_train, y_train)

In [50]:
y_pred = modelCB.predict(X_val)

In [51]:
recall_score(y_val, y_pred)

0.6163141993957704

In [92]:
y_pred_final = modelCB.predict(X_test)
recall_score(y_test, y_pred_final)

0.5950226244343891

In [65]:
cat_cols = X.columns[[7,8,9,10,11,12,13]]
cat_cols

Index(['Month', 'OperatingSystems', 'Browser', 'Region', 'TrafficType',
       'VisitorType', 'Weekend'],
      dtype='object')

### 2.4.2 Desicion Tree and GradientBoosting Pipeline

In [67]:
!pip install category_encoders -q

In [68]:
from category_encoders.one_hot import OneHotEncoder
from sklearn.pipeline import Pipeline

In [69]:
p1 = Pipeline([
    ('encoder_', OneHotEncoder(cols=cat_cols)),
    ('model_', DecisionTreeClassifier())
    ])

In [71]:
p2 = Pipeline([
    ('encoder_', OneHotEncoder(cols=cat_cols)),
    ('model_', GradientBoostingClassifier())
    ])

In [73]:
for i,p in enumerate([p1,p2]):
    p.fit(X_train, y_train)
    pred = p.predict(X_val)
    print(i+1, recall_score(y_val, pred))

1 0.6253776435045317
2 0.6283987915407855


In [74]:
params = {'max_depth' : np.arange(2,20),
          'min_samples_split' : np.arange(2,20,2),
          'min_samples_leaf': np.arange(1,20,2)}

p3 = Pipeline([
    ('encoder_', OneHotEncoder(cols=cat_cols)),
    ('model_', GridSearchCV(DecisionTreeClassifier(), params, cv=3, scoring='recall', verbose=0))
    ])

In [75]:
p3.fit(X_train, y_train)
pred = p3.predict(X_val)
recall_score(y_val, pred)

0.5649546827794562

In [77]:
p3.get_params()

{'memory': None,
 'steps': [('encoder_',
   OneHotEncoder(cols=Index(['Month', 'OperatingSystems', 'Browser', 'Region', 'TrafficType',
          'VisitorType', 'Weekend'],
         dtype='object'))),
  ('model_',
   GridSearchCV(cv=3, estimator=DecisionTreeClassifier(),
                param_grid={'max_depth': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
          19]),
                            'min_samples_leaf': array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19]),
                            'min_samples_split': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18])},
                scoring='recall'))],
 'verbose': False,
 'encoder_': OneHotEncoder(cols=Index(['Month', 'OperatingSystems', 'Browser', 'Region', 'TrafficType',
        'VisitorType', 'Weekend'],
       dtype='object')),
 'model_': GridSearchCV(cv=3, estimator=DecisionTreeClassifier(),
              param_grid={'max_depth': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
     

In [78]:
model = GridSearchCV(DecisionTreeClassifier(), params, cv=3, scoring='recall', verbose=0)

In [79]:
encoder = OneHotEncoder(cols=cat_cols)

In [81]:
X_test_1 = encoder.fit_transform(X_test)

In [83]:
X_val_1 = encoder.transform(X_val)

In [84]:
model.fit(X_test_1, y_test)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19]),
                         'min_samples_leaf': array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19]),
                         'min_samples_split': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18])},
             scoring='recall')

In [85]:
model.best_score_, model.best_params_

(0.5791812220383649,
 {'max_depth': 11, 'min_samples_leaf': 11, 'min_samples_split': 6})

In [87]:
y_pred = model.predict(X_val_1)
recall_score(y_val, y_pred)

0.5468277945619335

In [93]:
pred_final_DT = p1.predict(X_test)
recall_score(y_test, pred_final_DT)

0.5882352941176471

In [94]:
pred_final_GB = p2.predict(X_test)
recall_score(y_test, pred_final_GB)

0.5882352941176471

In [88]:
preds = p2.predict(X_val)
accuracy_score(y_val, preds)

0.8978388998035364

In [95]:
y_pred_final = modelCB.predict(X_test)
accuracy_score(y_test, y_pred_final)

0.8931466470154753

Mod Default Grid
CB  0.595   -
DT  0.588   0.565
GB  0.588   -

# 3. Explainer Dashboard

In [1]:
!pip install explainerdashboard -q

  You can safely remove it manually.


In [11]:
from explainerdashboard import ClassifierExplainer, ExplainerDashboard

In [12]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [19]:
explainer = ClassifierExplainer(modelCB, X_test, y_test, 
                                    model_output='logodds', labels=['Not Buy', 'Buy'])
                                    # cats=['Month', 'OperatingSystems', 'Browser', 'Region', 'TrafficType', 'VisitorType', 'Weekend'],
                                    # idxs=test_names, #names of passengers 
                                    

Generating self.shap_explainer = shap.TreeExplainer(model)


In [20]:
ExplainerDashboard(explainer, mode='external').run(8052)

Building ExplainerDashboard..
For this type of model and model_output interactions don't work, so setting shap_interaction=False...
The explainer object has no decision_trees property. so setting decision_trees=False...
Generating layout...
Calculating shap values...


C:\Users\maxle\AppData\Local\pypoetry\Cache\virtualenvs\eda-and-devs-Znx4-9sc-py3.11\Lib\site-packages\dash\dash.py:538: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



Calculating prediction probabilities...
Calculating metrics...
Calculating confusion matrices...
Calculating classification_dfs...
Calculating roc auc curves...
Calculating pr auc curves...
Calculating liftcurve_dfs...
Calculating dependencies...
Calculating permutation importances (if slow, try setting n_jobs parameter)...
Calculating pred_percentiles...
Calculating predictions...
Reminder: you can store the explainer (including calculated dependencies) with explainer.dump('explainer.joblib') and reload with e.g. ClassifierExplainer.from_file('explainer.joblib')
Registering callbacks...
Starting ExplainerDashboard on http://192.168.1.43:8052
You can terminate the dashboard with ExplainerDashboard.terminate(8052)


TypeError: 'NoneType' object cannot be interpreted as an integer

In [22]:
explainer.to_yaml("dashboard.yaml", explainerfile="explainer.joblib")